## GSAT trend patterns

In [1]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [2]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [3]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [4]:
variable_names = ['10yr', '30yr', '60yr']

In [5]:
# out put the forced trend and pvalue
dir_in ='/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/Disentangling_trend_analysis/Validation/ACCESS/trend/'

trend_annual_da = {}
p_val_annual_da = {}

for var in variable_names:
    trend_annual_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_forced_' + var + '_trend.nc', chunks={'run': 1})
    p_val_annual_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_forced_' + var + '_pvalue.nc', chunks={'run': 1})


In [6]:
for var in variable_names:
    trend_annual_da[var] = trend_annual_da[var].rename({'__xarray_dataarray_variable__': 'trend'})
    p_val_annual_da[var] = p_val_annual_da[var].rename({'__xarray_dataarray_variable__': 'p_val'})

In [7]:
# input the unforced trend and pvalue
unforced_trend_da = {}
unforced_p_val_da = {}

for var in variable_names:
    unforced_trend_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_unforced_' + var + '_trend.nc', chunks={'run': 1})
    unforced_p_val_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_unforced_' + var + '_pvalue.nc', chunks={'run': 1})

In [8]:
unforced_trend_da

{'10yr': <xarray.Dataset>
 Dimensions:                        (run: 37, lat: 90, lon: 180)
 Coordinates:
   * run                            (run) int64 1 2 3 4 5 6 ... 32 33 34 35 36 37
   * lat                            (lat) float64 -89.0 -87.0 -85.0 ... 87.0 89.0
   * lon                            (lon) float64 0.0 2.0 4.0 ... 356.0 358.0
 Data variables:
     __xarray_dataarray_variable__  (run, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>,
 '30yr': <xarray.Dataset>
 Dimensions:                        (run: 37, lat: 90, lon: 180)
 Coordinates:
   * run                            (run) int64 1 2 3 4 5 6 ... 32 33 34 35 36 37
   * lat                            (lat) float64 -89.0 -87.0 -85.0 ... 87.0 89.0
   * lon                            (lon) float64 0.0 2.0 4.0 ... 356.0 358.0
 Data variables:
     __xarray_dataarray_variable__  (run, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>,
 '60yr': <xarray.Dataset>
 Dimensions:        

In [9]:
for var in variable_names:
    unforced_trend_da[var] = unforced_trend_da[var].rename({'__xarray_dataarray_variable__': 'trend'})
    unforced_p_val_da[var] = unforced_p_val_da[var].rename({'__xarray_dataarray_variable__': 'p_val'})

### calculate the pattern correlation

In [10]:
# Input the 10-year, 30-year and 60-year ensemble mean SAT trend data
dir_input = '/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/Disentangling_trend_analysis/Figure4/data/'

# ACCESS_10yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_10yr_trend.nc')
# ACCESS_30yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_30yr_trend.nc')
# ACCESS_60yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_forced_60yr_trend.nc')

In [11]:
# ACCESS_10yr_trend = ACCESS_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
# ACCESS_30yr_trend = ACCESS_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
# ACCESS_60yr_trend = ACCESS_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

In [12]:
trend_annual_da['10yr'].trend.isel(run=1)

<xarray.DataArray 'trend' (lat: 90, lon: 180)>
dask.array<getitem, shape=(90, 180), dtype=float64, chunksize=(90, 180), chunktype=numpy.ndarray>
Coordinates:
    run      int64 2
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0

In [13]:
# pattern correlation betwenn observed forced pattern vs. Model simulated forced pattern
import scipy.stats as stats
# 10yr forced trend in each realization .vs. ACCESS ensemble trend ACCESS_10yr_trend
ACCESS_forced_10yr_trend = trend_annual_da['10yr'].trend
ACCESS_forced_30yr_trend = trend_annual_da['30yr'].trend
ACCESS_forced_60yr_trend = trend_annual_da['60yr'].trend

ACCESS_unforced_10yr_trend = unforced_trend_da['10yr'].trend
ACCESS_unforced_30yr_trend = unforced_trend_da['30yr'].trend
ACCESS_unforced_60yr_trend = unforced_trend_da['60yr'].trend

# # Pattern correlations calculation
# trend_pattern_correlation_10yr = []
# for i in range(len(trend_annual_da['10yr'].run)):
#     trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_forced_10yr_trend.isel(run=i).values.flatten(), ACCESS_10yr_trend['tas'].values.flatten())[0])

# trend_pattern_correlation_30yr = []
# for i in range(len(trend_annual_da['30yr'].run)):
#     trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_forced_30yr_trend.isel(run=i).values.flatten(), ACCESS_30yr_trend['tas'].values.flatten())[0])

# trend_pattern_correlation_60yr = []
# for i in range(len(trend_annual_da['60yr'].run)):
#     trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_forced_60yr_trend.isel(run=i).values.flatten(), ACCESS_60yr_trend['tas'].values.flatten())[0])
    

In [14]:
# trend_pattern_correlation_60yr

In [15]:
# # save the pattern correlation into text file
# # Save to a text file
# with open('pattern_correlations.txt', 'w') as file:
#     file.write('10-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_10yr:
#         file.write(f"{correlation}\n")

#     file.write('\n30-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_30yr:
#         file.write(f"{correlation}\n")

#     file.write('\n60-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_60yr:
#         file.write(f"{correlation}\n")

In [16]:
# Input the observational forced and unforced trend wrt. ACCESS-CM6A-LR
dir_obs = '/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/Disentangling_trend_analysis/Figure2/data/'

Obs_forced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_10yr_trend.nc')
Obs_forced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_30yr_trend.nc')
Obs_forced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_60yr_trend.nc')

Obs_unforced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_10yr_trend.nc')
Obs_unforced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_30yr_trend.nc')
Obs_unforced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_60yr_trend.nc')

In [17]:
Obs_unforced_10yr_trend

<xarray.Dataset>
Dimensions:                        (lat: 90, lon: 180)
Coordinates:
  * lat                            (lat) float64 -89.0 -87.0 -85.0 ... 87.0 89.0
  * lon                            (lon) float64 0.0 2.0 4.0 ... 356.0 358.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [18]:
Obs_forced_10yr_trend = Obs_forced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_30yr_trend = Obs_forced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_60yr_trend = Obs_forced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

Obs_unforced_10yr_trend = Obs_unforced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_30yr_trend = Obs_unforced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_60yr_trend = Obs_unforced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})


In [19]:
# calculate the pattern correlation between observed and model's each realization simulated forced and unforced trend
OBS_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_forced_10yr_trend.isel(run=i).values.flatten(), Obs_forced_10yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_forced_30yr_trend.isel(run=i).values.flatten(), Obs_forced_30yr_trend['tas'].values.flatten())[0])

OBS_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_forced_60yr_trend.isel(run=i).values.flatten(), Obs_forced_60yr_trend['tas'].values.flatten())[0])

In [20]:
OBS_trend_pattern_correlation_10yr

[0.6654576347029645,
 0.6823873531070208,
 0.4367509132972234,
 0.6540805384870976,
 0.5985313393785848,
 0.4988448004100973,
 0.7055658014086674,
 0.5515298082454735,
 0.5733516469746957,
 0.7128369640905068,
 0.6629073845877668,
 0.5976443196503664,
 0.6031430141459351,
 0.6412659879252163,
 0.6683108282847195,
 0.6845547957343265,
 0.5910289398244162,
 0.6438655712726926,
 0.6856007839895472,
 0.6159299972107671,
 0.685088066714706,
 0.6783183792740951,
 0.5911706678229606,
 0.6188346690069657,
 0.5693855192351466,
 0.6920725902300188,
 0.6470170592290299,
 0.7090070455201031,
 0.6119177731386428,
 0.46939067486355934,
 0.6155904275991201,
 0.695647979639985,
 0.5969838796217974,
 0.6219328306034957,
 0.6399660471092214,
 0.706566266279995,
 0.6722050081829549]

In [21]:
# Save to a text file
with open('obs_MMEM_forced_vs_ACCESS_forced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [22]:
OBS_unforced_trend_pattern_correlation_10yr = []
for i in range(len(trend_annual_da['10yr'].run)):
    OBS_unforced_trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_unforced_10yr_trend.isel(run=i).values.flatten(), Obs_unforced_10yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_30yr = []
for i in range(len(trend_annual_da['30yr'].run)):
    OBS_unforced_trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_unforced_30yr_trend.isel(run=i).values.flatten(), Obs_unforced_30yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_60yr = []
for i in range(len(trend_annual_da['60yr'].run)):
    OBS_unforced_trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_unforced_60yr_trend.isel(run=i).values.flatten(), Obs_unforced_60yr_trend['tas'].values.flatten())[0])

In [23]:
OBS_unforced_trend_pattern_correlation_30yr

[0.02455900849394744,
 0.10119037624229002,
 -0.3424955961440426,
 -0.17986455673758125,
 0.3074425903589431,
 -0.06332194679069984,
 -0.1322588179076079,
 0.16803593206198458,
 -0.16761392988417922,
 -0.09933307553582515,
 -0.2420037551883059,
 -0.03014511463342117,
 0.005951039883356217,
 0.16504080732409887,
 -0.30187618856000825,
 0.277922234771967,
 -0.051335944165469244,
 -0.4273387127669234,
 -0.1714238693395384,
 -0.22337720838650418,
 -0.2930591872750606,
 0.33820281394220986,
 0.2504201793252444,
 0.07415206017760931,
 0.059455274381378535,
 0.16681344412515084,
 0.2349164580972903,
 -0.21281604572374593,
 0.09161836797923077,
 -0.3929805212315532,
 -0.05239968393555731,
 -0.0644054472878112,
 -0.04001278895777408,
 0.05779124306035277,
 -0.4059913035293228,
 -0.11784156181444738,
 0.1347584263688143]

In [24]:
# Save to a text file
with open('obs_MMEM_unforced_vs_ACCESS_unforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

### Plotting with the Robinson Projections

In [ ]:
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.transparent'] = True # save the figure with a transparent background
x = np.arange(1850, 2015, 1)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
def plot_trend_with_significance(trend_data, lats, lons, p_values, GMST_p_values=None, levels=None, extend=None, cmap=None, title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """
    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()

    # Determine significance mask (where p-values are less than 0.05)
    insignificance_mask = p_values >= 0.10
    # Plotting
    # contour_obj = ax.pcolormesh(lons, lats, trend_data,  cmap='RdBu_r',vmin=-5.0, vmax=5.0, transform=ccrs.PlateCarree(central_longitude=180), shading='auto')
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    # Plot significance masks with different hatches
    ax.contourf(lons, lats, insignificance_mask, levels=[0.0, 0.10, 1.5],hatches=[None,'///'], colors='none', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      colors='gray', alpha=0.5, linestyle='--', linewidth=0.5)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj

In [ ]:
lat = trend_annual_da['10yr']['lat']
lon = trend_annual_da['10yr']['lon']

In [ ]:
lat

In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

In [ ]:
trend_annual_da['10yr']

In [ ]:
# title is from realization 1 to realization 21
titles = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 
          'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21',
            'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31',
            'r32', 'r33', 'r34', 'r35', 'r36', 'r37', 'r38', 'r39', 'r40', 'r41',
            'r42', 'r43', 'r44', 'r45', 'r46', 'r47', 'r48', 'r49', 'r50']
trend_annual_data = trend_annual_da['60yr'].trend*10.0
pvalue_annual_data = p_val_annual_da['60yr'].p_val

levels = np.arange(-0.6, 0.65, 0.05)
extend = 'both'

In [ ]:
type(trend_annual_data)

In [ ]:
print(trend_annual_data.dims)
print(pvalue_annual_data.dims)

In [ ]:
# check the min and max value of the trend
print(trend_annual_data.min().values)
print(trend_annual_data.max().values)

In [ ]:
# Plot the trend patterns 
def plot_data(ax, title):
    run_number = int(title.lstrip('r')) # Strip 'r' and convert to integer if necessary
    print(run_number)
    
    trend_data = trend_annual_data.sel({'run': run_number}) 
    p_values = pvalue_annual_data.sel({'run': run_number})
    
    trend_data_with_cyclic, lon_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
    p_values_with_cyclic, _ = cutil.add_cyclic_point(p_values, coord=lon)
    
    ax.set_global()
    
    # Assuming plot_trend_with_significance is defined correctly and returns a contour object
    contour_obj = plot_trend_with_significance(trend_data_with_cyclic, lat, lon_cyclic,
                                               p_values_with_cyclic,
                                               levels=levels, extend=extend,
                                               cmap='twilight_shifted', title=" ",
                                               ax=ax, 
                                               show_xticks=False, 
                                               show_yticks=False)
    ax.text(0.5, 1.08, title, fontsize=10, ha='center', va='center', transform=ax.transAxes)  # Adjusted to use transform
    ax.set_xticks([])
    ax.set_yticks([])
    return contour_obj

# Prepare for the loop
fig = plt.figure(figsize=(12, 15))
gs = gridspec.GridSpec(10, 5, wspace=0.1, hspace=0.01)

# Loop through and create subplots
for i, title in enumerate(titles):
    ax = fig.add_subplot(gs[i // 5, i % 5], projection=ccrs.Robinson(180))
    contour_obj=plot_data(ax, title)
    if i == 0:  # Arbitrarily choose the first plot's contour_obj for the colorbar
        contour_obj_for_cbar = contour_obj
        
# Assuming contour_obj is defined and consistent across plots
# Add horizontal colorbar
cbar_ax = fig.add_axes([0.25, 0.06, 0.5, 0.012])  # Adjust these values as needed
cbar = plt.colorbar(contour_obj, cax=cbar_ax, orientation='horizontal', extend='both')
cbar.ax.tick_params(labelsize=14)
cbar.set_label('Annual SAT Trend (°C/decade)', fontsize=16)

plt.tight_layout()
fig.savefig('ACCESS-1950-2022-60year-forced-trend-Patterns-sig90%.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# client.close()
# scluster.close()